# Using Google Vision AI
This is a document that outlines steps to using Google's Vision AI API for local photos.

## Requirements
- A Google Cloud account
- Billing Information (Google claims it will not auto charge to card if doing Free Trial. The Free trial includes $300 credit and lasts up to a year).
- Python 3

## Creating Google Cloud account
1) Create Google Cloud information by creating a Google Cloud account "Get Started for Free" or "Sign In" through your Google account.
![photo](1.png")

2) Enable API through https://console.cloud.google.com/ by clicking on the 3 bars icon > "API & Services" which will open to a new webpage.

![photo](2.png")

3) Click on "+ Enable APIs and Services" and search for "Cloud Vision API"

![photo](3.png")![photo](4.png")

4) Click the blue "Enable API" button (The screenshot shows up as "Manage" because it is already enabled")
![photo](5.png")

## Using Google Cloud Vision API
Before beginning you have to installl Google Cloud SDK, Create a Project, Enable Billing, <b>and</b> retrieve your personal google application credentials in order to use the API. Keep it secret. 

See https://cloud.google.com/sdk/install as it explains the steps for installation in detail.

Some packages to import are: `io`, `os`, `google.cloud`, and `google.cloud.vision`

Recommended to import `pandas` for data wrangling.

In [79]:
import io
import os
import pandas as pd
from IPython.display import Image, HTML
from google.cloud import vision
from google.cloud.vision import types

Put in your google credentials:

In [34]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "/Users/SusanneGov/Documents/My First Project-bcfca3ee21b0.json" # replace with your own 
client = vision.ImageAnnotatorClient()

To grab information from a collection of photos:

In [59]:
photo_folder = os.getcwd() + r'/SignPhotos' # your photo folder file path
photo_info = {} # to put in text annotations
label_info = {} # to put in labels
object_info = {} # to put in multiple objects

# goes through photo folder and 
for dirpath, subdirs, files in os.walk(photo_folder):
    for x in files:
        if x.endswith(".png"): # can be jpeg
            photo_path = os.path.join(dirpath, x)
            with io.open(photo_path, 'rb') as image_file:
                content = image_file.read()
            image = types.Image(content=content) 
            texts = client.text_detection(image=image)
            texts = texts.text_annotations
            labels = client.label_detection(image=image)
            labels = labels.label_annotations
            objects = client.object_localization(image=image).localized_object_annotations
            object_info[x] = [object_.name for object_ in objects]
            label_info[x] =  [label.description for label in labels]
            photo_info[x] =  str(texts[0].description).split("\n") #creates list of text identified

Create pandas dataframe to display results.

In [75]:
words = {}
labels = {}
objects = {}
for keys in photo_info:
    words[keys] = [x.upper() for x in photo_info[keys]]
    labels[keys] = [x.upper() for x in label_info[keys]]
    objects[keys] = [x.upper() for x in object_info[keys]]
words = pd.DataFrame.from_dict(words,orient='index').sort_index()
labels = pd.DataFrame.from_dict(labels,orient='index').sort_index()
objects = pd.DataFrame.from_dict(objects,orient='index').sort_index()
def path_to_image_html(path):
    return '<img src="'+ path + '"/>'

In [80]:
display(words.head()) #Shows texts in photos identified

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
104_N-checkpoint.png,CONNECTED,PHARMACY CARE,GOES BEYOND,PRESCRIPTIONS.,LEFT TURN,YIELD,ON GREEN,LAMAR,RD,PARKWAY 1200,...,None,None,None,None,None,None,None,None,None,None
104_N.png,CONNECTED,PHARMACY CARE,GOES BEYOND,PRESCRIPTIONS.,LEFT TURN,YIELD,ON GREEN,LAMAR,- ENFIELD RD,PARKWAY 1200,...,None,None,None,None,None,None,None,None,None,None
104_S-checkpoint.png,PARKWAY 1200,ENFIELD RD,THE T VERN,"OPEN ""GLASS",CONTAINES,PROHUBIT,GOOGLE,IMAGE CAPTURE: MAR 2019 0 2020 GOOGLE UNITED S...,REPORT A PROBLEM,,...,None,None,None,None,None,None,None,None,None,None
104_S.png,PARKWAY,RD,ENFIELD,1200,LAMAR,STHE T VERN,TAVERN,"OPEN ""GLASS",CONTAINEPS,PROHUBIT,...,None,None,None,None,None,None,None,None,None,None
111_E-checkpoint 2.png,LEFT TURN,YIELD,N GREEN,URGE,KIN,JEFFERSON,(3500 ST 3400>,IN FOWER CO,AUSTINFLOWERCO.,HOME,...,None,None,None,None,None,None,None,None,None,None


In [77]:
display(labels)

,0,1,2,3,4,5,6,7,8,9
104_N-checkpoint.png,ROAD,TRAFFIC LIGHT,SIGNALING DEVICE,LANE,HIGHWAY,THOROUGHFARE,LIGHTING,MOTOR VEHICLE,TRANSPORT,INFRASTRUCTURE
104_N.png,ROAD,TRAFFIC LIGHT,SIGNALING DEVICE,LANE,LIGHTING,TRANSPORT,MOTOR VEHICLE,THOROUGHFARE,HIGHWAY,LIGHT FIXTURE
104_S-checkpoint.png,TRAFFIC LIGHT,SIGNALING DEVICE,LIGHTING,URBAN AREA,LIGHT FIXTURE,TRANSPORT,SIGNAGE,TREE,STREET LIGHT,STREET
104_S.png,TRAFFIC LIGHT,SIGNALING DEVICE,LIGHTING,LIGHT FIXTURE,URBAN AREA,SIGNAGE,ROAD,STREET LIGHT,TREE,SIGN
111_E-checkpoint 2.png,ROAD,LIGHTING,TRAFFIC LIGHT,LANE,SIGNALING DEVICE,HIGHWAY,LIGHT FIXTURE,METROPOLITAN AREA,INFRASTRUCTURE,STREET
111_E-checkpoint.png,ROAD,LIGHTING,TRAFFIC LIGHT,LANE,SIGNALING DEVICE,RESIDENTIAL AREA,METROPOLITAN AREA,STREET,TRANSPORT,THOROUGHFARE
111_E.png,ROAD,HIGHWAY,TRAFFIC LIGHT,LANE,SIGNALING DEVICE,LIGHTING,INFRASTRUCTURE,STREET,RESIDENTIAL AREA,THOROUGHFARE
111_N-checkpoint.png,TRAFFIC LIGHT,SIGNALING DEVICE,ROAD,LIGHTING,STREET SIGN,LIGHT FIXTURE,STREET LIGHT,OVERHEAD POWER LINE,TRANSPORT,THOROUGHFARE
111_N.png,TRAFFIC LIGHT,SIGNALING DEVICE,LIGHTING,ROAD,STREET SIGN,LIGHT FIXTURE,TRANSPORT,STREET LIGHT,THOROUGHFARE,LANE
111_S-checkpoint.png,RESIDENTIAL AREA,TRANSPORT,METROPOLITAN AREA,ARCHITECTURE,BUILDING,LIGHTING,NEIGHBOURHOOD,URBAN AREA,SUBURB,SKY


In [78]:
display(objects)

,0,1,2,3,4,5
104_N-checkpoint.png,CAR,CAR,None,None,None,None
104_N.png,CAR,CAR,None,None,None,None
104_S-checkpoint.png,LIGHTING,None,None,None,None,None
104_S.png,LIGHTING,STREET LIGHT,None,None,None,None
111_E-checkpoint 2.png,STREET LIGHT,STREET LIGHT,BUILDING,None,None,None
111_E-checkpoint.png,LIGHTING,STREET LIGHT,None,None,None,None
111_E.png,STREET LIGHT,STREET LIGHT,LIGHTING,None,None,None
111_N-checkpoint.png,CAR,CAR,CAR,CAR,None,None
111_N.png,CAR,CAR,CAR,CAR,None,None
111_S-checkpoint.png,BUILDING,None,None,None,None,None
